In [1]:
# cd /Users/dhiveshakilan/Learning/Python/AI/IntelliAISupport
# python3.12 -m venv IntelliAISupport
# source IntelliAISupport/bin/activate

In [2]:
!pip3 install langchain
!pip3 install -U langchain-community
!pip3 install openai
!pip3 install PyPDF2
!pip3 install faiss-cpu
!pip3 install tiktoken
!pip3 install Flask

In [3]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [4]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from flask import Flask, request, jsonify, render_template
import base64
import requests
import smtplib
from email.mime.text import MIMEText
import uuid
import openai
import json
import re
import os

In [5]:
with open('/Users/dhiveshakilan/Learning/Python/AI/IntelliAISupport/key.json') as f:
    key=json.load(f)

os.environ["OPENAI_API_KEY"] = key['API_KEY'] 


In [6]:
file_path= '/Users/dhiveshakilan/Learning/Python/AI/IntelliAISupport/Info Doc/'

# List of PDF files
pdf_files = ['AMGEO.pdf','Eliot.pdf','Error.pdf','Ole_feeding.pdf','role_access.pdf', 'gop_access.pdf']  # Add more PDF paths as needed

# Concatenate all the content from multiple PDFs
raw_text = ''

# Loop through each PDF file


In [7]:
for pdf_file in pdf_files:
    pdf = file_path+pdf_file
    pdfreader = PdfReader(pdf)

    # Read and extract text from each page in the PDF
    for i, page in enumerate(pdfreader.pages):
        content = page.extract_text()
        if content:
            raw_text += content

# Now 'raw_text' contains all the concatenated text from multiple PDF files
print(raw_text)


AMGEO:AMGEO batch running on dev124 —> dev124 outbound files created —> outbound folder—>sent to VM dev89 via for—> this file will be present in outbound folder of dev89 —>CSSS process send the file to CTM —> Match processing done in CTM —> CTM sends file with matched details the VM —> File will be present in inbound folder of VM —> FTR batch sends this file from VM to inbound folder of dev124 —> this inbound file is pocked by the AMGEO batch.Overview about Eliot Application: • Eliot is the main and official application for Equity and Derivative deals management.  • Predominantly, Eliot manages Middle Office functions for Deal capture & Enrichment, Deal Life Cycle Management, position management etc.  • It helps in providing data to other Risk & Accounting applications and provide deal data to BO Application for further processing and settlements.Solution for user error messages:anupriyam has No Modification Habilitation on PRO/TSF Flow. -> If user encounters this error then it means n

In [8]:
raw_text

'AMGEO:AMGEO batch running on dev124 —> dev124 outbound files created —> outbound folder—>sent to VM dev89 via for—> this file will be present in outbound folder of dev89 —>CSSS process send the file to CTM —> Match processing done in CTM —> CTM sends file with matched details the VM —> File will be present in inbound folder of VM —> FTR batch sends this file from VM to inbound folder of dev124 —> this inbound file is pocked by the AMGEO batch.Overview about Eliot Application: • Eliot is the main and official application for Equity and Derivative deals management.  • Predominantly, Eliot manages Middle Office functions for Deal capture & Enrichment, Deal Life Cycle Management, position management etc.  • It helps in providing data to other Risk & Accounting applications and provide deal data to BO Application for further processing and settlements.Solution for user error messages:anupriyam has No Modification Habilitation on PRO/TSF Flow. -> If user encounters this error then it means 

In [9]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [10]:
len(texts)

1

In [11]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

/var/folders/yy/bjx87s_s3_j5r1yvqr9nvt680000gn/T/ipykernel_27710/939171391.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [12]:
document_search = FAISS.from_texts(texts, embeddings)

In [13]:
document_search


In [14]:
chain = load_qa_chain(OpenAI(temperature = 0.5), chain_type="stuff")

/var/folders/yy/bjx87s_s3_j5r1yvqr9nvt680000gn/T/ipykernel_27710/474082722.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  chain = load_qa_chain(OpenAI(temperature = 0.5), chain_type="stuff")
/var/folders/yy/bjx87s_s3_j5r1yvqr9nvt680000gn/T/ipykernel_27710/474082722.py:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/doc

In [15]:
app = Flask(__name__)

gop_data = [
    {
        "DPS": [
            {"Portfolio": "PDS_ABC", "Status": "Active"},
            {"Portfolio": "PDS_XYZ", "Status": "Active"},
            {"Portfolio": "PDS_DEF", "Status": "Active"},
            {"Portfolio": "PDS_DEY", "Status": "Not Active"}
        ],
        "XYZ": [
            {"Portfolio": "XYZ_ABC", "Status": "Active"}
        ],
        "FX" : [
            {"Portfolio": "FX-GY-NOMGT", "Status": "Active"}
        ],
        "SNIKO" : [
            {"Portfolio": "SN-FX-GY", "Status": "Active"}
        ]
    }
]

roles_data = [
    {"Role": "DELETE_FLUX_X", "Description": "Delete flows via excel", "Keywords": "delete, flows, excel, not allowed to delete flows, no permission for deleting flows"},
    {"Role": "DELETE_OLD_DEAL_X", "Description": "Massive delete old deal x", "Keywords": "delete, old deals, massive delete, no permission for deleting deals, delete deal, not allowed to delete deals"},
    {"Role": "MAJ_FLOW_CVA", "Description": "Input of CVA Flows", "Keywords": "CVA, CVA flows, no access for CVA flows, unable to input CVA data"},
    {"Role": "MAJ_FLOW_OPERATIONEL", "Description": "Input of AJT, CLR, MTM Flows", "Keywords": "AJT flows, CLR flows, MTM flows, no permission for AJT, no access for CLR, unable to input MTM data, AJT, CLR, MTM"},
    {"Role": "MAJ_FLOW_TRESORESULT", "Description": "Input of FOR, RES, SAF, FIN, EMB, EMD Flows", "Keywords": "treasury result flows, FOR, RES, SAF, FIN, EMB, EMD, no permission for treasury flows, not allowed to input FOR data"},
    {"Role": "MAJ_FLOW_TSF_SHB", "Description": "Input of TSF/SHB Flows", "Keywords": "TSF, TSF flows, SHB flows, no access to TSF/SHB, unable to input TSF/SHB data"},
    {"Role": "RESULTAT_VALO", "Description": "Input of RHE, REM Flows", "Keywords": "RHE, RHE flows, REM flows, not allowed to input RHE/REM flows, no access for RHE/REM flows"},
    {"Role": "MAJ_FLOW_FLX", "Description": "Input of FLX Flows", "Keywords": "FLX, FLX flows, no permission for FLX flows, not allowed to input FLX data"},
    {"Role": "MAJ_FLOW_WHT", "Description": "Input of WHT Flows", "Keywords": "WHT, WHT flows, no access to WHT flows, unable to input WHT data"}
]

counterpart_data = []

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def langchain_doc(query):
    docs = document_search.similarity_search(query)
    r=chain.run(input_documents=docs, question=query)
    return r

# Function to match user query with role keywords using partial matching
def match_role(role, roles_data):
    for i in range(len(roles_data)):
        if role.trim() in roles_data[i]['Role']:
            return roles_data[i]['Role'],roles_data[i]['Description']
    return "No matching role found.", "No description available"

def find_gop_using_ptf(user_query, gop_data):
    found_gop=None
    for gop_dict in gop_data:
        for gop_name, portfolios in gop_dict.items():
            for portfolio_info in portfolios:
                if portfolio_info["Portfolio"] in user_query:
                    # found_gop = {"GOP": gop_name, "Portfolio": portfolio_info}
                    found_gop = gop_name if portfolio_info["Status"] == "Active" else f"{portfolio_info["Portfolio"]} - Portfolio not active"
                    break
    return found_gop if found_gop is not None else "No Portfolio found."

def match_gop(user_query, gop_data):
    user_query = re.findall(r'\b\w+\b', user_query)
    print(user_query)
    for gop in user_query:
        if gop in gop_data[0].keys():
            return gop
    return "No matching gop found."

def counterpart(user_query, counterpart_data):
    # No counterpart Found
    return

# You have not the authorization to perform this action. (technical authorizationn: RUN with discrimnant 'SNIKO' on process group 'EVENT PROCESSING')

def generate_request_id():
    return str(uuid.uuid4())

@app.route('/send_email', methods = ['POST'])
def send_approval_email():
    sender_email = "dhivesh18122000@gmail.com"  # Replace with your email
    sender_password = 'stwf byvp tuhv lskz'
    role = request.form['role']
    user_email = request.form['userEmail']
    manager_email = request.form['managerEmail']
    # confirmation_message = request.form['confirmationMessage']
    request_id = generate_request_id()
    subject = f"Approval Request for {role} - Reference ID: {request_id}"
    
    body = f'''Hello,
        User {user_email} is requesting access to the role or GOP: {role}.
        The request ID for this approval is {request_id}.
        Please review and provide your approval.
        Sincerely, 
        Eliot Team'''
    
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = manager_email

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.send_message(msg)
        return jsonify({'status': 'success', 'message': 'Approval email sent successfully.'})
    
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)})

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/image_to_text', methods=['POST'])
def extract_and_match():
    if 'image' not in request.files:
        return jsonify({"error": "No image part found"}), 400
    
    file = request.files['image']
    
    if file.filename == '':
        return jsonify({"error": "No image file selected"}), 400
    
    # Save the file temporarily
    file_path = '/tmp/' + file.filename
    file.save(file_path)
    
    base64_image = encode_image(file_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.getenv("OPENAI_API_KEY")}"
    }

    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Extract only the error message text from this image."
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    if response.status_code == 429:
        return jsonify({"error": "Rate limit exceeded. Please try again later."}), 429

    if response.status_code != 200:
        return jsonify({"error": f"API request failed with status code {response.status_code}", "details": response.json()}), response.status_code

    try:
        response_json = response.json()
        user_query = response_json['choices'][0]['message']['content']
    except KeyError:
        return jsonify({"error": "Failed to parse response", "response": response.json()}), 500

    if not user_query:
        return jsonify({"error": "No message extracted"}), 400
# query = "anupriyam has No Modification Habilitation on MAJ_FLOW_CVA Flow."

    query = "Why I am getting this message - "+user_query 
    r=langchain_doc(query)
# q = f"Only return the portfolio name from this response: {r}"
    
    if 'role' in r.lower():
        q = f"Only return the role name from this response: {r}"
        matching_role, description = match_role(langchain_doc(q),roles_data)
    elif 'Portfolio' in user_query:
        matching_role = find_gop_using_ptf(user_query, gop_data)
        response_message = f"Matching Portfolio GOP: {matching_role}"
    elif 'Counterpart' in user_query.lower():
        matching_role = counterpart(user_query, counterpart_data)
        response_message = f"Eliot Code is: {matching_role}"
    else:
        matching_role = match_gop(user_query, gop_data)
        response_message = f"Matching GOP: {matching_role}"

    return jsonify({
        "extracted_message": user_query,
        "response_message": response_message
    })

@app.route('/text_to_role', methods=['POST'])
def text_to_role():
    text_message = request.form.get('textMessage', '')
    
    if not text_message:
        return jsonify({"error": "No text message provided"}), 400
    
    query = "Why I am getting this message - "+text_message 
    r=langchain_doc(query)
# q = f"Only return the portfolio name from this response: {r}"
    
    if 'role' in r.lower():
        q = f"Only return the role name from this response: {r}"
        matching_role, description = match_role(langchain_doc(q),roles_data)
    elif 'Portfolio' in text_message.lower():
        matching_role = find_gop_using_ptf(text_message, gop_data)
        response_message = f"Matching Portfolio GOP: {matching_role}"
    elif 'Counterpart' in text_message.lower():
        matching_role = counterpart(text_message, counterpart_data)
        response_message = f"Eliot Code is: {matching_role}"
    else:
        matching_role = match_gop(text_message, gop_data)
        response_message = f"Matching GOP: {matching_role}"

    return jsonify({
        "extracted_message": text_message,
        "response_message": response_message
    })
if __name__ == '__main__':
    app.run(debug=True)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/dhiveshakilan/Library/Python/3.12/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/dhiveshakilan/Library/Python/3.12/lib/python/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/Users/dhiveshakilan/Library/Python/3.12/lib/python/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/dhiveshakilan/Library/Python/3.12/lib/python/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/Users/dhiveshakilan/Library/Python/3.12/lib/python/site-packages/ipykernel/kernelapp.py", line 331, in i

SystemExit: 1

/Users/dhiveshakilan/Library/Python/3.12/lib/python/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [27]:
# # query = "You aren’t allowed to do select with portfolio = SN-FX-GY"
# query = "anupriyam has No Modification Habilitation on MAJ_FLOW_CVA Flow."
# docs = document_search.similarity_search(query)
# r=chain.run(input_documents=docs, question=query)
# print(r)
# # q = f"Only return the portfolio name from this response: {r}"
# q = f"Only return the role name from this response: {r}"
# print(q)
# chain.run(input_documents=docs, question=q)

/var/folders/yy/bjx87s_s3_j5r1yvqr9nvt680000gn/T/ipykernel_26517/3475382305.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  r=chain.run(input_documents=docs, question=query)


 If user encounters this error, it means they do not have access to the MAJ_FLOW_CVA Flow. To access this, you can add this role in their specific user role: MAJ_FLOW_CVA.
Only return the role name from this response:  If user encounters this error, it means they do not have access to the MAJ_FLOW_CVA Flow. To access this, you can add this role in their specific user role: MAJ_FLOW_CVA.


' MAJ_FLOW_CVA'

In [28]:
# query = "explain the amgeo flow"
# docs = document_search.similarity_search(query)
# chain.run(input_documents=docs, question=query)

' The AMGEO flow involves a batch running on the dev124 server, creating outbound files in the outbound folder, which are then sent to the VM dev89 via a for process. The file will be present in the outbound folder of dev89, and then the CSSS process will send the file to CTM. The file will undergo match processing in CTM, and then CTM will send the file with matched details to the VM. The file will be present in the inbound folder of the VM, and then the FTR batch will send this file from the VM to the inbound folder of dev124. Finally, the AMGEO batch will pick up the inbound file for further processing. '

In [29]:
# query = "give me ole process job name and job details"
# docs = document_search.similarity_search(query)
# chain.run(input_documents=docs, question=query)

' The OLE process job name is ELTC-ELI-OLE and it runs every 30 minutes. The shell script used for the job is eli_ole.sh and it runs on the dev124 server. '